In [1]:
import sys
import classWrapTools
import fisherTools
import pickle
from mpi4py import MPI
import scipy
import numpy
import os

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.rcParams.update({"text.usetex": True,"font.family": "serif"})

#MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print(rank, size)

0 1


In [2]:
###  Set of experiments  ###
expNames = list(range(1))
nExps = len(expNames)
lmax = 5000
lmaxTT = 5000
lmin = 2
noiseLevels = [1]
beamSizeArcmin = 1.4

lbuffer = 1500
lmax_calc = lmax+lbuffer

expNamesThisNode = numpy.array_split(numpy.asarray(expNames), size)[rank]

classExecDir = './CLASS_delens/'
classDataDir = './CLASS_delens/'

classDataDirThisNode = classDataDir + 'data/Node_' + str(rank) + '/'
fileBase = 'ExpB_correlation'
fileBaseThisNode = fileBase + '_' + str(rank)

if not os.path.exists(classDataDirThisNode):
    os.makedirs(classDataDirThisNode)

spectrumTypes = ['unlensed', 'lensed', 'delensed', 'lensing']
polCombs = ['cl_TT', 'cl_TE', 'cl_EE', 'cl_BB', 'cl_dd']

#######################################################################################3
#LOAD PARAMS AND GET POWER SPECTRA

#Fiducial values and step sizes taken from arXiv:1509.07471 Allison et al
cosmoFid = {'omega_c_h2':0.1197, \
                'omega_b_h2': 0.0222, \
                'N_eff': 3.046, \
                'A_s' : 2.196e-9, \
                'n_s' : 0.9655,\
                'tau' : 0.06, \
                #'H0' : 67.5, \
                'theta_s' : 0.010409, \
                #'Yhe' : 0.25, \
                #'r'   : 0.01, \
                'mnu' : 0.06}
#cosmoFid['n_t'] = - cosmoFid['r'] / 8.0 * (2.0 - cosmoFid['n_s'] - cosmoFid['r'] / 8.0)

stepSizes = {'omega_c_h2':0.0030, \
                'omega_b_h2': 0.0008, \
                'N_eff': .080, \
                'A_s' : 0.1e-9, \
                'n_s' : 0.010,\
                'tau' : 0.020, \
                'H0' : 1.2, \
                'theta_s' : 0.000050, \
                'mnu' : 0.02, \
                #'r'   : 0.001, \
                #'n_t' : cosmoFid['n_t'], \
                'Yhe' : 0.0048}

cosmoParams = list(cosmoFid.keys())
ell = numpy.arange(2,lmax_calc+1+2000)

reconstructionMask = dict()
reconstructionMask['lmax_T'] = 3000

extra_params = dict()
#extra_params['delensing_verbose'] = 3
#extra_params['output_spectra_noise'] = 'no'
#extra_params['write warnings'] = 'y'

ellsToUse = {'cl_TT': [lmin, lmaxTT], 'cl_TE': [lmin, lmax], 'cl_EE': [lmin, lmax], 'cl_dd': [2, lmax]}
ellsToUseNG = {'cl_TT': [lmin, lmaxTT], 'cl_TE': [lmin, lmax], 'cl_EE': [lmin, lmax], 'cl_dd': [2, lmax], 'lmaxCov': lmax_calc}

cmbNoiseSpectra = dict()
deflectionNoises = dict()
powersFid = dict()

doNonGaussian = True
includeUnlensedSpectraDerivatives = True

### Assign task of computing lensed NG covariance to last node       ###
### This is chosen because last node sometimes has fewer experiments ###
if doNonGaussian is True:
    if rank == size-1:

        if includeUnlensedSpectraDerivatives:
            dCldCLd_lensed, dCldCLu_lensed = classWrapTools.class_generate_data(cosmoFid,
                                         cmbNoise = None, \
                                         deflectionNoise = None, \
                                         extraParams = extra_params, \
                                         rootName = fileBaseThisNode, \
                                         lmax = lmax_calc, \
                                         calculateDerivatives = 'lensed', \
                                         includeUnlensedSpectraDerivatives = includeUnlensedSpectraDerivatives,
                                         classExecDir = classExecDir,
                                         classDataDir = classDataDirThisNode)
        else:
            dCldCLd_lensed = classWrapTools.class_generate_data(cosmoFid,
                                         cmbNoise = None, \
                                         deflectionNoise = None, \
                                         extraParams = extra_params, \
                                         rootName = fileBaseThisNode, \
                                         lmax = lmax_calc, \
                                         calculateDerivatives = 'lensed', \
                                         classExecDir = classExecDir,
                                         classDataDir = classDataDirThisNode)
            dCldCLu_lensed = None

        print('Successfully computed derivatives')
        #stop
    else:
        dCldCLd_lensed = None

for k in expNamesThisNode:
    expName = expNames[k]

    print('Node ' + str(rank) + ' working on experiment ' + str(expName))

    cmbNoiseSpectra[k] = classWrapTools.noiseSpectra(l = ell,
                                                noiseLevelT = noiseLevels[k],
                                                useSqrt2 = True,
                                                beamArcmin = beamSizeArcmin)

    powersFid[k], deflectionNoises[k] = classWrapTools.class_generate_data(cosmoFid,
                                         cmbNoise = cmbNoiseSpectra[k],
                                         extraParams = extra_params,
                                         rootName = fileBaseThisNode,
                                         lmax = lmax_calc,
                                         classExecDir = classExecDir,
                                         classDataDir = classDataDirThisNode,
                                         reconstructionMask = reconstructionMask)

    if doNonGaussian:

        ### Overwrite dCldCLd_delensed for each experiment to save memory ###

        if includeUnlensedSpectraDerivatives:
            dCldCLd_delensed, dCldCLu_delensed = classWrapTools.class_generate_data(cosmoFid,
                                                 cmbNoise = cmbNoiseSpectra[k], \
                                                 deflectionNoise = deflectionNoises[k], \
                                                 extraParams = extra_params, \
                                                 rootName = fileBaseThisNode, \
                                                 lmax = lmax_calc, \
                                                 calculateDerivatives = 'delensed', \
                                                 includeUnlensedSpectraDerivatives = includeUnlensedSpectraDerivatives,
                                                 classExecDir = classExecDir,
                                                 classDataDir = classDataDirThisNode)
        else:
            dCldCLd_delensed = classWrapTools.class_generate_data(cosmoFid,
                                                 cmbNoise = cmbNoiseSpectra[k], \
                                                 deflectionNoise = deflectionNoises[k], \
                                                 extraParams = extra_params, \
                                                 rootName = fileBaseThisNode, \
                                                 lmax = lmax_calc, \
                                                 calculateDerivatives = 'delensed', \
                                                 classExecDir = classExecDir,
                                                 classDataDir = classDataDirThisNode)
            dCldCLu_delensed = None

        ############################
        ## Seems to hang on bcast ##
        ############################

        if rank != size-1 and dCldCLd_lensed is None:
            classDataDirLastNode = classDataDir + 'data/Node_' + str(size-1) + '/'
            fileBaseLastNode = fileBase + '_' + str(size-1)

            dCldCLd_lensed = classWrapTools.loadLensingDerivatives(rootName = fileBaseLastNode,
                                                                   classDataDir = classDataDirLastNode,
                                                                   dervtype = 'lensed')


            dCldCLu_lensed = None
            if includeUnlensedSpectraDerivatives:
                dCldCLu_lensed = classWrapTools.loadUnlensedSpectraDerivatives(rootName = fileBaseLastNode,
                                                                   classDataDir = classDataDirLastNode,
                                                                   dervtype = 'lensed')

        myColors = 'PRGn'
        ticksShort = [0,200,400,600,800,1000]
        
        sTypes = ['$C_{\ell}^{TT}$', '$C_{\ell}^{TE}$', '$C_{\ell}^{EE}$', \
                  '$C_{\ell}^{BB}$', '$C_{\ell}^{dd}$']
        nPlots = len(sTypes)
        
        ticks = [0,1000,2000,3000,4000,5000]

        myMin = -0.001
        myMax = 0.001
        
        fs = 4
        padjust = 0.12
        
        corrMatrix_delensed = fisherTools.getNonGaussianCov(powersFid = powersFid[k], \
                                cmbNoiseSpectra = cmbNoiseSpectra[k], \
                                deflectionNoiseSpectra = deflectionNoises[k], \
                                dCldCLd = dCldCLd_delensed, \
                                dCldCLu = dCldCLu_delensed, \
                                lmax = 5000, \
                                polCombsToUse = polCombs, \
                                spectrumType = 'delensed',\
                                rescaleToCorrelation = True)
        
        myMatrix = corrMatrix_delensed
        subLength = int(len(myMatrix)/(nPlots))
        plt.figure('covmats', figsize=(8,10))
             
        for i in range(nPlots-1):
            for j in range(nPlots):
                if i <= j:
                    ax = plt.subplot(nPlots, nPlots-1, (nPlots-1)*j + i + 1)
                    
                    if j==nPlots-1:
                        plt.imshow(myMatrix[j*subLength:j*subLength+1000,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax, aspect='auto')
                    else:
                        plt.imshow(myMatrix[j*subLength:(j+1)*subLength,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax)
                    
                    ax.tick_params(direction="in",top=True,right=True)
                    if i==0:
                        plt.ylabel(sTypes[j],fontsize=22+fs,labelpad=-2.0)
                        if j==nPlots-1:
                            plt.yticks(ticksShort,fontsize=12+fs)
                        else:
                            plt.yticks(ticks,fontsize=12+fs)
                    else:
                        if j==nPlots-1:
                            plt.yticks(ticksShort,[])
                        else:
                            plt.yticks(ticks,[])
                    if j==nPlots-1:
                        plt.xlabel(sTypes[i],fontsize=22+fs,labelpad=10.0)
                        plt.xticks(ticks,rotation='vertical',fontsize=12+fs)
                    else:
                        plt.xticks(ticks,[])
        
        axins = inset_axes(ax,
                           width="15%",
                           height="100%",
                           loc='upper right',
                           bbox_to_anchor=(-0.6,3.5,1,2),
                           bbox_transform=ax.transAxes,
                           borderpad=0
                           )
        cbar = plt.colorbar(cax=axins)
        cbar.remove()
        plt.tight_layout(pad=0.4)
        plt.subplots_adjust(left=padjust,bottom=padjust)
        plt.savefig(fileBase+'_delensed.pdf')
        plt.close()
        
        #off
        corrMatrix_delensed_off = fisherTools.getNonGaussianCov(powersFid = powersFid[k], \
                                cmbNoiseSpectra = cmbNoiseSpectra[k], \
                                deflectionNoiseSpectra = deflectionNoises[k], \
                                dCldCLd = dCldCLd_delensed, \
                                dCldCLu = None, \
                                lmax = 5000, \
                                polCombsToUse = polCombs, \
                                spectrumType = 'delensed',\
                                rescaleToCorrelation = True)
        
        #diff
        corrMatrix_delensed_diff = corrMatrix_delensed - corrMatrix_delensed_off
        
        myMatrix = corrMatrix_delensed_diff
        plt.figure('covmats', figsize=(8,10))
        
        for i in range(nPlots-1):
            for j in range(nPlots):
                if i <= j:
                    ax = plt.subplot(nPlots, nPlots-1, (nPlots-1)*j + i + 1)
                    
                    if j==nPlots-1:
                        plt.imshow(myMatrix[j*subLength:j*subLength+1000,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax, aspect='auto')
                    else:
                        plt.imshow(myMatrix[j*subLength:(j+1)*subLength,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax)
                    
                    ax.tick_params(direction="in",top=True,right=True)
                    if i==0:
                        plt.ylabel(sTypes[j],fontsize=22+fs,labelpad=-2.0)
                        if j==nPlots-1:
                            plt.yticks(ticksShort,fontsize=12+fs)
                        else:
                            plt.yticks(ticks,fontsize=12+fs)
                    else:
                        if j==nPlots-1:
                            plt.yticks(ticksShort,[])
                        else:
                            plt.yticks(ticks,[])
                    if j==nPlots-1:
                        plt.xlabel(sTypes[i],fontsize=22+fs,labelpad=10.0)
                        plt.xticks(ticks,rotation='vertical',fontsize=12+fs)
                    else:
                        plt.xticks(ticks,[])
        
        axins = inset_axes(ax,
                           width="15%",
                           height="100%",
                           loc='upper right',
                           bbox_to_anchor=(-0.8,3.5,1,2),
                           bbox_transform=ax.transAxes,
                           borderpad=0
                           )
        cbar = plt.colorbar(cax=axins)
        cbar.ax.tick_params(direction="in",labelsize=12+fs)
        plt.tight_layout(pad=0.4)
        plt.subplots_adjust(left=padjust,bottom=padjust)
        plt.savefig(fileBase+'_delensed_diff.pdf')
        plt.close()
        
        ############################
        ## Seems to hang on bcast ##
        ############################
        
#         if rank != size-1 and dCldCLd_lensed is None:
#             classDataDirLastNode = classDataDir + 'data/Exp_' + str(size-1) + '/'
#             fileBaseLastNode = fileBase + '_' + str(size-1)

#             dCldCLd_lensed = classWrapTools.loadLensingDerivatives(rootName = fileBaseLastNode,
#                                                                    classDataDir = classDataDirLastNode,
#                                                                    dervtype = 'lensed')


#             dCldCLu_lensed = None
#             if includeUnlensedSpectraDerivatives:
#                 dCldCLu_lensed = classWrapTools.loadUnlensedSpectraDerivatives(rootName = fileBaseLastNode,
#                                                                    classDataDir = classDataDirLastNode,
#                                                                    dervtype = 'lensed')

        
        corrMatrix_lensed = fisherTools.getNonGaussianCov(powersFid = powersFid[k], \
                                cmbNoiseSpectra = cmbNoiseSpectra[k], \
                                deflectionNoiseSpectra = deflectionNoises[k], \
                                dCldCLd = dCldCLd_lensed, \
                                dCldCLu = dCldCLu_lensed, \
                                lmax = 5000, \
                                polCombsToUse = polCombs, \
                                spectrumType = 'lensed',\
                                rescaleToCorrelation = True)
        
        myMatrix = corrMatrix_lensed
        plt.figure('covmats', figsize=(8,10))
        
        for i in range(nPlots-1):
            for j in range(nPlots):
                if i <= j:
                    ax = plt.subplot(nPlots, nPlots-1, (nPlots-1)*j + i + 1)
                    
                    if j==nPlots-1:
                        plt.imshow(myMatrix[j*subLength:j*subLength+1000,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax, aspect='auto')
                    else:
                        plt.imshow(myMatrix[j*subLength:(j+1)*subLength,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax)
                    
                    ax.tick_params(direction="in",top=True,right=True)
                    if i==0:
                        plt.ylabel(sTypes[j],fontsize=22+fs,labelpad=-2.0)
                        if j==nPlots-1:
                            plt.yticks(ticksShort,fontsize=12+fs)
                        else:
                            plt.yticks(ticks,fontsize=12+fs)
                    else:
                        if j==nPlots-1:
                            plt.yticks(ticksShort,[])
                        else:
                            plt.yticks(ticks,[])
                    if j==nPlots-1:
                        plt.xlabel(sTypes[i],fontsize=22+fs,labelpad=10.0)
                        plt.xticks(ticks,rotation='vertical',fontsize=12+fs)
                    else:
                        plt.xticks(ticks,[])
        
        axins = inset_axes(ax,
                           width="15%",
                           height="100%",
                           loc='upper right',
                           bbox_to_anchor=(-0.6,3.5,1,2),
                           bbox_transform=ax.transAxes,
                           borderpad=0
                           )
        cbar = plt.colorbar(cax=axins)
        cbar.remove()
        plt.tight_layout(pad=0.4)
        plt.subplots_adjust(left=padjust,bottom=padjust)
        plt.savefig(fileBase+'_lensed.pdf')
        plt.close()
        
        #off
        corrMatrix_lensed_off = fisherTools.getNonGaussianCov(powersFid = powersFid[k], \
                                cmbNoiseSpectra = cmbNoiseSpectra[k], \
                                deflectionNoiseSpectra = deflectionNoises[k], \
                                dCldCLd = dCldCLd_lensed, \
                                dCldCLu = None, \
                                lmax = 5000, \
                                polCombsToUse = polCombs, \
                                spectrumType = 'lensed',\
                                rescaleToCorrelation = True)
        
        #diff
        corrMatrix_lensed_diff = corrMatrix_lensed - corrMatrix_lensed_off
        
        myMatrix = corrMatrix_lensed_diff
        plt.figure('covmats', figsize=(8,10))
        
        for i in range(nPlots-1):
            for j in range(nPlots):
                if i <= j:
                    ax = plt.subplot(nPlots, nPlots-1, (nPlots-1)*j + i + 1)
                    
                    if j==nPlots-1:
                        plt.imshow(myMatrix[j*subLength:j*subLength+1000,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax, aspect='auto')
                    else:
                        plt.imshow(myMatrix[j*subLength:(j+1)*subLength,i*subLength:(i+1)*subLength],cmap=myColors,\
                                  vmin = myMin, vmax = myMax)
                    
                    ax.tick_params(direction="in",top=True,right=True)
                    if i==0:
                        plt.ylabel(sTypes[j],fontsize=22+fs,labelpad=-2.0)
                        if j==nPlots-1:
                            plt.yticks(ticksShort,fontsize=12+fs)
                        else:
                            plt.yticks(ticks,fontsize=12+fs)
                    else:
                        if j==nPlots-1:
                            plt.yticks(ticksShort,[])
                        else:
                            plt.yticks(ticks,[])
                    if j==nPlots-1:
                        plt.xlabel(sTypes[i],fontsize=22+fs,labelpad=10.0)
                        plt.xticks(ticks,rotation='vertical',fontsize=12+fs)
                    else:
                        plt.xticks(ticks,[])
        
        axins = inset_axes(ax,
                           width="15%",
                           height="100%",
                           loc='upper right',
                           bbox_to_anchor=(-0.6,3.5,1,2),
                           bbox_transform=ax.transAxes,
                           borderpad=0
                           )
        cbar = plt.colorbar(cax=axins)
        cbar.remove()
        plt.tight_layout(pad=0.4)
        plt.subplots_adjust(left=padjust,bottom=padjust)
        plt.savefig(fileBase+'_lensed_diff.pdf')
        plt.close()

print('Node ' + str(rank) + ' finished all experiments')

Successfully computed derivatives
Node 0 working on experiment 0
Computing covaraince for  cl_TT  and  cl_TT
Adding covaraince from dcl_TT/dcl_TT and dcl_TT/dcl_TT
Computing covaraince for  cl_TE  and  cl_TT
Adding covaraince from dcl_TE/dcl_TE and dcl_TT/dcl_TT
Computing covaraince for  cl_TE  and  cl_TE
Adding covaraince from dcl_TE/dcl_TE and dcl_TE/dcl_TE
Computing covaraince for  cl_EE  and  cl_TT
Adding covaraince from dcl_EE/dcl_EE and dcl_TT/dcl_TT
Adding covaraince from dcl_EE/dcl_BB and dcl_TT/dcl_TT
Computing covaraince for  cl_EE  and  cl_TE
Adding covaraince from dcl_EE/dcl_EE and dcl_TE/dcl_TE
Adding covaraince from dcl_EE/dcl_BB and dcl_TE/dcl_TE
Computing covaraince for  cl_EE  and  cl_EE
Adding covaraince from dcl_EE/dcl_EE and dcl_EE/dcl_EE
Adding covaraince from dcl_EE/dcl_EE and dcl_EE/dcl_BB
Adding covaraince from dcl_EE/dcl_BB and dcl_EE/dcl_EE
Adding covaraince from dcl_EE/dcl_BB and dcl_EE/dcl_BB
Computing covaraince for  cl_BB  and  cl_TT
Adding covaraince from

/hpc/applications/anaconda/3/lib/python3.6/site-packages/matplotlib/figure.py:2299: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


Computing covaraince for  cl_TT  and  cl_TT
Adding covaraince from dcl_TT/dcl_TT and dcl_TT/dcl_TT
Computing covaraince for  cl_TE  and  cl_TT
Adding covaraince from dcl_TE/dcl_TE and dcl_TT/dcl_TT
Computing covaraince for  cl_TE  and  cl_TE
Adding covaraince from dcl_TE/dcl_TE and dcl_TE/dcl_TE
Computing covaraince for  cl_EE  and  cl_TT
Adding covaraince from dcl_EE/dcl_EE and dcl_TT/dcl_TT
Adding covaraince from dcl_EE/dcl_BB and dcl_TT/dcl_TT
Computing covaraince for  cl_EE  and  cl_TE
Adding covaraince from dcl_EE/dcl_EE and dcl_TE/dcl_TE
Adding covaraince from dcl_EE/dcl_BB and dcl_TE/dcl_TE
Computing covaraince for  cl_EE  and  cl_EE
Adding covaraince from dcl_EE/dcl_EE and dcl_EE/dcl_EE
Adding covaraince from dcl_EE/dcl_EE and dcl_EE/dcl_BB
Adding covaraince from dcl_EE/dcl_BB and dcl_EE/dcl_EE
Adding covaraince from dcl_EE/dcl_BB and dcl_EE/dcl_BB
Computing covaraince for  cl_BB  and  cl_TT
Adding covaraince from dcl_BB/dcl_EE and dcl_TT/dcl_TT
Adding covaraince from dcl_BB/dc